In [154]:
from openai import OpenAI
from dotenv import load_dotenv
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import os
from urllib.parse import urlparse, urljoin
import json

In [155]:
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
client = OpenAI()

In [156]:
class Website_link:
    url: str
    title: str
    text: str
    internal_links: list

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        self.title = soup.title.string if soup.title else "No title found"
        
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        
        self.text = soup.body.get_text(separator="\n", strip=True)
        
        self.internal_links = []
        domain = urlparse(url).netloc
        for a in soup.find_all('a', href=True):
            link = urljoin(url, a['href'])  
            if urlparse(link).netloc == domain:  
                self.internal_links.append(link)

In [157]:
class Website_text:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [188]:

import json

weblist = ['https://techcrunch.com/', 'https://www.theverge.com/',
           'https://www.wired.com/', 'https://arstechnica.com/',
           'https://gizmodo.com/']
final_text = {}

%timeit
for site in weblist:
    print(f"Loading... {site}")
    ed = Website_link(site)
    internal_links = f'{ed.internal_links}'

    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[
            {
                'role': 'system',
                'content': (
                    'You will return only the links which contain '
                    'a news headline. You will return only the top 3 '
                    'news that you consider relevant in tech. You will '
                    'return each of the headlines and the links in a '
                    'python dict. You will only return the dict without the '
                    'python format. The python dict needs to be in this '
                    'sample format: {"1": {"headline":"...","link":"..."},"2": {"headline":"...","link":"..."}}'
                )
            },
            {'role': 'user', 'content': internal_links}
        ]
    )

    links = response.choices[0].message.content
    links = json.loads(links)

    results = []

    for i in links:
        result = Website_text(links[i]['link'])
        result_text = result.text

        response = client.chat.completions.create(
            model='gpt-4o-mini',
            messages=[
                {
                    'role': 'system',
                    'content': (
                        'This is a newspaper news. You will resume the '
                        'text into 1 paragraph. You will write a 3 minutes '
                        'text talking about this news like you are a YouTuber '
                        'news presenter. You will return the resume paragraph '
                        'and the 3 minutes text in a python dict "resume" and "text". You will only return the dict without the '
                        'python format.'
                    )
                },
                {'role': 'user', 'content': result_text}
            ]
        )

        response_text = json.loads(response.choices[0].message.content)
        results.append({links[i]['headline']: [response_text, {'url': links[i]['link']}]})

    for item in results:
        final_text.update(item)



Loading... https://techcrunch.com/
Loading... https://www.theverge.com/
Loading... https://www.wired.com/
Loading... https://arstechnica.com/
Loading... https://gizmodo.com/


In [189]:
for title, details in final_text.items():
    display(Markdown(f'<h1>{title}</h1>'))
    print(f"Source: <{details[1]['url']}>")
    
    if details:
        detail = details[0]
        
        display(Markdown("<h4>Summary</h4>"))
        print()
        print(detail['resume'])
        print()
        
        display(Markdown("<h4>Suggested Text to Say</h4>"))
        print()
        print(detail['text'])
        print()

<h1>Jon McNeill's insights on Tesla's robotaxi and why EV startups fail</h1>

Source: <https://techcrunch.com/2024/10/27/jon-mcneills-insights-on-teslas-robotaxi-and-why-ev-startups-fail/>


<h4>Summary</h4>


Jon McNeill, former Tesla and Lyft executive, shared insights at the World Business Forum about Tesla's automation ambitions for its robotaxi, emphasizing a surprising lack of substantial developments during its recent reveal. He questioned Tesla's reliance solely on cameras for self-driving capabilities, suggesting that lidar technology might be necessary for safety. Moreover, he discussed the broader challenges facing EV startups, noting that strong leadership is essential for navigating the complexities of car manufacturing, as evidenced by the struggles of companies like Fisker and Rivian.



<h4>Suggested Text to Say</h4>

...

Source: <https://gizmodo.com/meet-the-99-ton-robot-thats-printing-rocket-labs-next-gen-neutron-rocket-2000514991>


<h4>Summary</h4>


Rocket Lab has unveiled a groundbreaking 99-ton, 39-foot tall custom-built 3D printing robot at its Maryland manufacturing complex, which is set to significantly speed up the production of its Neutron rocket by laying down carbon composite materials at a remarkable speed. This world-first automated fiber placement machine is designed to construct the largest structures of the Neutron rocket, including its upper stage domes and fuel tanks, potentially saving over 15,000 hours of manufacturing time. Rocket Lab CEO Peter Beck expressed excitement for the innovative technology, which will not only be used for the Neutron but also for other projects, positioning Rocket Lab to compete with industry giants in a rapidly evolving aerospace sector.



<h4>Suggested Text to Say</h4>


Hey everyone! Welcome back to the channel! Today we’re diving into some exciting news from the world of space technology. Rocket Lab, a key player in the aerospace industry, has just rolled out a colossal 99-ton, 39-foot robot that could redefine how rockets are built! This isn't just any robot; it's the largest of its kind in the world and is capable of 3D printing components for Rocket Lab's upcoming Neutron rocket. Imagine this, folks: this beast can lay down carbon fiber composite layers at a jaw-dropping speed of 328 feet per minute! That's like finishing a task that would typically take weeks in just ONE day! Talk about efficiency! 

So, what does this mean for Rocket Lab? Well, the Neutron rocket is designed to be a medium-lift launch vehicle with a reusable first stage, allowing it to deliver hefty payloads to low Earth orbit. Thanks to this cutting-edge 3D printer, Rocket Lab is aiming to save a whopping 15,000 hours in manufacturing time! Rocket Lab's CEO, Peter Beck, is buz